In [1]:
import os
os.environ['PROJ_LIB'] = '/data/keeling/a/sshu3/anaconda2/share/proj'
from mpl_toolkits.basemap import Basemap, cm, shiftgrid,maskoceans
from netCDF4 import Dataset as NetCDFFile
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib import cm
import numpy.ma as ma
from scipy.interpolate import griddata
from pyresample import geometry,image, kd_tree
import mycolorbars
import math


In [7]:
# Read in the other products
#nclu=NetCDFFile('rcp45_9100_oxid.nc','r')
nclu=NetCDFFile('/data/jain1/c/sshu3/SBGC/results/CONUS/decadal_mean/rcp85_91_00_noslr.nc','r')
ncvar_net = nclu.variables['ch4_flux_wet'][:]+nclu.variables['ch4_flux_inund'][:]
ncvar_oxid = nclu.variables['ch4_oxid_wet'][:]+nclu.variables['ch4_oxid_inund'][:]
ncvar_prod = nclu.variables['ch4_prod_wet'][:]+nclu.variables['ch4_prod_inund'][:]
ncvar_ebul = nclu.variables['ch4_ebul_wet'][:]+nclu.variables['ch4_ebul_inund'][:]
ncvar_aere = nclu.variables['ch4_aere_wet'][:]+nclu.variables['ch4_aere_inund'][:]
ncvar_diff = ncvar_net - ncvar_ebul - ncvar_aere
#ncvar_diff[ncvar_diff<-0.5]= 0.1
latnc = nclu.variables['lat'][:]
# znc = nclu.variables['level'][:]
lonnc = nclu.variables['lon'][:]
# timenc = nclu.variables['time'][:]
lonnc = ((lonnc - 180) % 360) - 180

nclu_fw=NetCDFFile('/data/jain1/c/sshu3/SBGC/results/get_us_tot/fw_frac_mean_01_10.nc','r')
#nclu3=NetCDFFile('rcp45_soiloxid.nc','r')
fw = nclu_fw.variables['FW'][:]

nclu2=NetCDFFile('/data/jain1/c/sshu3/SBGC/results/CONUS/decadal_mean/rcp85_91_00.nc','r')
ncvar2_net = nclu2.variables['ch4_flux_wet'][:]+nclu2.variables['ch4_flux_inund'][:]
ncvar2_oxid = nclu2.variables['ch4_oxid_wet'][:]+nclu2.variables['ch4_oxid_inund'][:]
ncvar2_prod = nclu2.variables['ch4_prod_wet'][:]+nclu2.variables['ch4_prod_inund'][:]
ncvar2_ebul = nclu2.variables['ch4_ebul_wet'][:]+nclu2.variables['ch4_ebul_inund'][:]
ncvar2_aere = nclu2.variables['ch4_aere_wet'][:]+nclu2.variables['ch4_aere_inund'][:]
ncvar2_diff = ncvar2_net - ncvar2_ebul - ncvar2_aere
#ncvar2_diff[ncvar2_diff<-0.5]= 0.1
nclu_fw2=NetCDFFile('/data/jain1/c/sshu3/SBGC/results/get_us_tot/fw_frac_mean_91_00_rcp45.nc','r')
#nclu3=NetCDFFile('rcp45_soiloxid.nc','r')
fw2 = nclu_fw2.variables['FW'][:]

# nclu3=NetCDFFile('/data/jain1/c/sshu3/SBGC/results/CONUS/decadal_mean/rcp85_91_00.nc','r')
# ncvar3_net = nclu3.variables['ch4_flux'][:]
# ncvar3_oxid = nclu3.variables['ch4_oxid'][:]
# ncvar3_prod = nclu3.variables['ch4_prod'][:]
# ncvar3_ebul = nclu3.variables['ch4_ebul'][:]
# ncvar3_aere = nclu3.variables['ch4_aere'][:]
# ncvar3_diff = ncvar3_net - ncvar3_ebul - ncvar3_aere
# #ncvar3_diff[ncvar3_diff<-0.5]= 0.1
# nclu_fw3=NetCDFFile('/data/jain1/c/sshu3/SBGC/results/get_us_tot/fw_frac_mean_91_00_rcp85.nc','r')
# #nclu3=NetCDFFile('rcp45_soiloxid.nc','r')
# fw3 = nclu_fw3.variables['FW'][:]

# Open Coastal wetland mask
ncmk_c=NetCDFFile('/data/jain1/c/sshu3/SBGC/results/get_us_tot/surfdata_05x05.nc','r')
ncvar_mask_c = ncmk_c.variables['COASTMASK'][:]

# Mask out regions outside the CONUS
ncmk=NetCDFFile('/data/jain1/c/sshu3/SBGC/results/get_us_tot/surfdata_05x05_13reg.nc','r')
ncvar_mask = ncmk.variables['REGION_MASK_CRU_NCEP'][:]
#ncvar_mask[ncvar_mask<12] = 0
#ncvar_mask[ncvar_mask>12] = 0
#ncvar_mask[ncvar_mask==12] = 1

ncvar_net[ncvar_mask<12] = ma.masked  #ncvar_net*ncvar_mask
ncvar_net[ncvar_mask>12] = ma.masked  #ncvar_net*ncvar_mask
ncvar2_net[ncvar_mask<12] = ma.masked  #ncvar2_net*ncvar_mask
ncvar2_net[ncvar_mask>12] = ma.masked  #ncvar2_net*ncvar_mask
# ncvar3_net[ncvar_mask<12] = ma.masked  #ncvar3_net*ncvar_mask
# ncvar3_net[ncvar_mask>12] = ma.masked  #ncvar3_net*ncvar_mask

# # Only account coastal wetland
# ncvar_net[ncvar_mask_c==1] = ma.masked  #ncvar_net*ncvar_mask
# ncvar2_net[ncvar_mask_c==1] = ma.masked  #ncvar2_net*ncvar_mask

# # Only account inland wetland
# ncvar_net[ncvar_mask_c==2] = ma.masked  #ncvar_net*ncvar_mask
# ncvar2_net[ncvar_mask_c==2] = ma.masked  #ncvar2_net*ncvar_mask

ncvar_prod[ncvar_mask<12] = ma.masked  #ncvar_net*ncvar_mask
ncvar_prod[ncvar_mask>12] = ma.masked  #ncvar_net*ncvar_mask
ncvar2_prod[ncvar_mask<12] = ma.masked  #ncvar2_net*ncvar_mask
ncvar2_prod[ncvar_mask>12] = ma.masked  #ncvar2_net*ncvar_mask
# ncvar3_prod[ncvar_mask<12] = ma.masked  #ncvar3_net*ncvar_mask
# ncvar3_prod[ncvar_mask>12] = ma.masked  #ncvar3_net*ncvar_mask

ncvar_oxid[ncvar_mask<12] = ma.masked  #ncvar_net*ncvar_mask
ncvar_oxid[ncvar_mask>12] = ma.masked  #ncvar_net*ncvar_mask
ncvar2_oxid[ncvar_mask<12] = ma.masked  #ncvar2_net*ncvar_mask
ncvar2_oxid[ncvar_mask>12] = ma.masked  #ncvar2_net*ncvar_mask
# ncvar3_oxid[ncvar_mask<12] = ma.masked  #ncvar3_net*ncvar_mask
# ncvar3_oxid[ncvar_mask>12] = ma.masked  #ncvar3_net*ncvar_mask

ncvar_ebul[ncvar_mask<12] = ma.masked  #ncvar_net*ncvar_mask
ncvar_ebul[ncvar_mask>12] = ma.masked  #ncvar_net*ncvar_mask
ncvar2_ebul[ncvar_mask<12] = ma.masked  #ncvar2_net*ncvar_mask
ncvar2_ebul[ncvar_mask>12] = ma.masked  #ncvar2_net*ncvar_mask
# ncvar3_ebul[ncvar_mask<12] = ma.masked  #ncvar3_net*ncvar_mask
# ncvar3_ebul[ncvar_mask>12] = ma.masked  #ncvar3_net*ncvar_mask

ncvar_aere[ncvar_mask<12] = ma.masked  #ncvar_net*ncvar_mask
ncvar_aere[ncvar_mask>12] = ma.masked  #ncvar_net*ncvar_mask
ncvar2_aere[ncvar_mask<12] = ma.masked  #ncvar2_net*ncvar_mask
ncvar2_aere[ncvar_mask>12] = ma.masked  #ncvar2_net*ncvar_mask
# ncvar3_aere[ncvar_mask<12] = ma.masked  #ncvar3_net*ncvar_mask
# ncvar3_aere[ncvar_mask>12] = ma.masked  #ncvar3_net*ncvar_mask

ncvar_diff[ncvar_mask<12] = ma.masked  #ncvar_net*ncvar_mask
ncvar_diff[ncvar_mask>12] = ma.masked  #ncvar_net*ncvar_mask
ncvar2_diff[ncvar_mask<12] = ma.masked  #ncvar2_net*ncvar_mask
ncvar2_diff[ncvar_mask>12] = ma.masked  #ncvar2_net*ncvar_mask
# ncvar3_diff[ncvar_mask<12] = ma.masked  #ncvar3_net*ncvar_mask
# ncvar3_diff[ncvar_mask>12] = ma.masked  #ncvar3_net*ncvar_mask

#diff = ncvar2_net - ncvar_net
#ncvar2_net[abs(diff)>100] = np.float('nan')


In [9]:
# Calculate total
nlat = 360
nlon = 720

# Area of each grid
grid_area = np.zeros((nlat, nlon))
EARTH_AREA = 5.096e14
lat = np.arange(-89.75, 90.25, 0.5)
res = 0.5

for i in np.arange(0,nlat):
    for j in np.arange(0,nlon):
        grid_area[i,j] = (EARTH_AREA/2)*abs(np.sin((lat[i] - res/2)*math.pi/180) -
                np.sin((lat[i] + res/2)*math.pi/180))/(360/res)

# Read the 2D files, For ISAM-1D

# Calculate the global total CH4
totch4_array = ncvar_net*grid_area
totch4 = np.nansum(totch4_array)/1e12
print('The global methane ...')
print(totch4)

The global methane ...
26.029826245695634


In [ ]:
# Numbers for table
# Wetland
# Hist:
# N: 15.3
# A: 6.5
# E: 2.7
# D: 6.0
# P: 15.5
# O: 5.9e-10

# RCP45:
# N: 19.4
# A: 7.9
# E: 3.3
# D: 8.2
# P: 20.0
# O: 3.0e-10

# N_noslr: 19.9

# RCP85:
# N: 24.7
# A: 9.9
# E: 4.8
# D: 10.0
# P: 25.3
# O: 2.9e-10

# N_noslr: 26

# Non-wetland
# Hist:
# N: -1.5
# A: 0.7
# E: 0
# D: -2.1
# P: 10.0
# O: 11.7

# RCP45:
# N: -1.5
# A: 1.3
# E: 0
# D: -2.7
# P: 18.7
# O: 21.6

# RCP85:
# N: -2.1
# A: 1.8
# E: 0
# D: -3.9
# P: 27.4
# O: 32.2

# Total (Sum of both, no necessary to calculate again...)


In [ ]:
# Hist
# NEW Inland: 8.1
# OLD Inland: 8.1
# NEW Coastal: 7.2
# OLD Coastal: 14.1

# Old total: 22.2
# New total: 15.3

In [ ]:
# RCP4.5
# NEW Inland: 12.3
# OLD Inland: 12.3
# NEW Coastal: 7.1
# OLD Coastal: 15.2

# Old total: 27.5
# New total: 19.4

In [ ]:
# RCP8.5
# NEW Inland: 16.2
# OLD Inland: 16.2
# NEW Coastal: 8.5
# OLD Coastal: 19.7

# Old total: 35.9
# New total: 24.7

In [ ]:
22.3-4.4

In [ ]:
diff.count()

In [ ]:
# Check the Wetcharts file results

# My personal colormap
my_cmap = mycolorbars.color_ch4emission()

fig = plt.figure(figsize=(15,25))

#ax1 = fig.add_subplot(221)
ax1 = fig.add_subplot(311)
ax1.set_title("Net $CH_4$ flux (gC-$CH_4$/$m^2$/yr) ",fontsize=20)
#ax1.set_title("$CH_4$ Oxidation flux (gC-$CH_4$/$m^2$/yr) ",fontsize=20)
#ax1.set_title("$CH_4$ Production (gC-$CH_4$/$m^2$/yr)",fontsize=20)
#ax1.set_title("$CH_4$ Aerenchyma transport (gC-$CH_4$/$m^2$/yr)",fontsize=20)
#ax1.set_title("$CH_4$ Ebullition (gC-$CH_4$/$m^2$/yr)",fontsize=20)
#ax1.set_title("$CH_4$ Diffusion (gC-$CH_4$/$m^2$/yr)",fontsize=20)

map = Basemap(llcrnrlon=-119,llcrnrlat=23,urcrnrlon=-63,urcrnrlat=51,
        projection='lcc',lat_1=33,lat_2=45,lon_0=-95)
# draw coastlines, state and country boundaries, edge of map.
map.drawcoastlines()
map.drawstates()
map.drawcountries(color='b')
lon,lat = np.meshgrid(lonnc,latnc) #Returns coordinate matrices from coordinate vectors
x,y = map(lon,lat)
cs1 = map.pcolormesh(x,y,ncvar_net[:,:],
              norm=colors.SymLogNorm(linthresh=0.1, linscale=0.1, vmin=-1.0, vmax=100),cmap=my_cmap) # use log color scale,                   
#              norm=colors.LogNorm(vmin=0.01, vmax=100),cmap=plt.cm.jet) # use log color scale,
                                                      # omit this to use linear
                                                     # color scale
#cs1.cmap.set_under('w')
cbar = map.colorbar(cs1,location='bottom',pad="5%",extend='both')
#plt.title('Maize harvested area fraction')
cbar.ax.tick_params(labelsize=30) 

ax1 = fig.add_subplot(312)
ax1.set_title("Net $CH_4$ flux (gC-$CH_4$/$m^2$/yr) RCP4.5",fontsize=20)
#ax1.set_title("$CH_4$ Oxidation (gC-$CH_4$/$m^2$/yr) RCP4.5",fontsize=20)
#ax1.set_title("$CH_4$ Production (gC-$CH_4$/$m^2$/yr) RCP4.5",fontsize=20)
#ax1.set_title("$CH_4$ Aerenchyma transport (gC-$CH_4$/$m^2$/yr) RCP4.5",fontsize=20)
#ax1.set_title("$CH_4$ Ebullition (gC-$CH_4$/$m^2$/yr) RCP4.5",fontsize=20)
#ax1.set_title("$CH_4$ Diffusion (gC-$CH_4$/$m^2$/yr) RCP4.5",fontsize=20)

map = Basemap(llcrnrlon=-119,llcrnrlat=23,urcrnrlon=-63,urcrnrlat=51,
        projection='lcc',lat_1=33,lat_2=45,lon_0=-95)
# draw coastlines, state and country boundaries, edge of map.
map.drawcoastlines()
map.drawstates()
map.drawcountries(color='b')
lon,lat = np.meshgrid(lonnc,latnc) #Returns coordinate matrices from coordinate vectors
x,y = map(lon,lat)
cs = map.pcolormesh(x,y,ncvar2_net[:,:],
              norm=colors.SymLogNorm(linthresh=0.1, linscale=0.1, vmin=-1.0, vmax=100.0),cmap=my_cmap) # use log color scale,
#              norm=colors.LogNorm(vmin=0.01, vmax=100),cmap=plt.cm.jet) # use log color scale,
                                                      # omit this to use linear
                                                     # color scale
#cs.cmap.set_under('w')
cbar = map.colorbar(cs,location='bottom',pad="5%",extend='both')
cbar.ax.tick_params(labelsize=30) 

ax1 = fig.add_subplot(313)
ax1.set_title("Net $CH_4$ flux (gC-$CH_4$/$m^2$/yr) RCP8.5",fontsize=20)
#ax1.set_title("$CH_4$ Oxidation (gC-$CH_4$/$m^2$/yr) RCP8.5",fontsize=20)
#ax1.set_title("$CH_4$ Production (gC-$CH_4$/$m^2$/yr) RCP8.5",fontsize=20)
#ax1.set_title("$CH_4$ Aerenchyma transport (gC-$CH_4$/$m^2$/yr) RCP8.5",fontsize=20)
#ax1.set_title("$CH_4$ Ebullition (gC-$CH_4$/$m^2$/yr) RCP8.5",fontsize=20)
#ax1.set_title("$CH_4$ Diffusion (gC-$CH_4$/$m^2$/yr) RCP8.5",fontsize=20)

map = Basemap(llcrnrlon=-119,llcrnrlat=23,urcrnrlon=-63,urcrnrlat=51,
        projection='lcc',lat_1=33,lat_2=45,lon_0=-95)

# draw coastlines, state and country boundaries, edge of map.
map.drawcoastlines()
map.drawstates()
map.drawcountries(color='b')
lon,lat = np.meshgrid(lonnc,latnc) #Returns coordinate matrices from coordinate vectors
x,y = map(lon,lat)
cs = map.pcolormesh(x,y,ncvar3_net[:,:],
              norm=colors.SymLogNorm(linthresh=0.1, linscale=0.1, vmin=-1.0, vmax=100.0),cmap=my_cmap) # use log color scale,
#              norm=colors.LogNorm(vmin=0.01, vmax=100),cmap=plt.cm.jet) # use log color scale,
                                                      # omit this to use linear
                                                     # color scale
#cs.cmap.set_under('w')
cbar = map.colorbar(cs,location='bottom',pad="5%",extend='both')
cbar.ax.tick_params(labelsize=30) 

#plt.show()
#plt.savefig('Oxid_newbar_CH4.jpg',dpi=300)

In [ ]:
### An example of how to use make_cmap
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure()
ax = fig.add_subplot(311)
### Create a list of RGB tuples
colors = [(255,0,0), (255,255,0), (255,255,255), (0,157,0), (0,0,255)] # This example uses the 8-bit RGB
### Call the function make_cmap which returns your colormap
my_cmap = make_cmap(colors, bit=True)
### Use your colormap
plt.pcolor(np.random.rand(25,50), cmap=my_cmap)
plt.colorbar()

ax = fig.add_subplot(312)
colors = [(1,1,1), (0.5,0,0)] # This example uses the arithmetic RGB
### If you are only going to use your colormap once you can
### take out a step.
plt.pcolor(np.random.rand(25,50), cmap=make_cmap(colors))
plt.colorbar()

ax = fig.add_subplot(313)
colors = [(0.4,0.2,0.0), (1,1,1), (0,0.3,0.4)]
### Create an array or list of positions from 0 to 1.
position = [0, 0.3, 1]
plt.pcolor(np.random.rand(25,50), cmap=make_cmap(colors, position=position))
plt.colorbar()

plt.savefig("custom_cmap.png")
plt.show()

In [ ]:
plt.imshow(np.random.rand(10,10),cmap=cm.copper)
plt.show()